In [ ]:
## TODO: importation des modules necessaire aux traitements et aux analyses spatiales
#! ces modules sont dependants des uns des autres
import pandas as pd
import ee
import geedim
import numpy as np
import polars as pl
import geemap
from geemap.datasets import DATA
import geopandas as gpd
import geedim
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
Map = geemap.Map()
Map.add_basemap('SATELLITE')
# TODO: importation des shapefilepour la zone d'etude (shp,geojson, etc.)
roi = geemap.vector_to_ee("delim_4326.shp")

In [31]:
# ON a selectionné l'année plusieru temporealité pour obtenir une statistique descripitve des valeurs des bandes spectrales
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)
s2 = ee.ImageCollection(DATA.COPERNICUS_S2_SR_HARMONIZED).filterDate('2018-01-01', '2019-12-31').filterBounds(roi)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
# En utilisant la focntion size() on peut obtenir le nombre d'image dans l'image collection
print(f'le nombre dimage collection en selectionnant ladate, {s2.size().getInfo()}')


le nombre dimage collection en selectionnant ladate, 80


# Etape 2, creation des indices spectrales pour discriminer les types de couvertures

In [ ]:
# add variable in bands
inbands = ["B2","B3","B4","B8","B11"]
#Add spectral indices

def func_azz (image):
    ndvi=image.normalizedDifference(["B8","B4"]).rename(["NDVI"])
    ndwi=image.normalizedDifference(["B8","B11"]).rename(["NDWI"])
    evi= image.expression(
    '2.5 * ((NIR-RED) / (NIR + 6 * RED - 7.5* BLUE +1))',{
            "NIR":image.select("B8").divide(10000),
            "RED":image.select("B4").divide(10000),
            "BLUE":image.select("B2").divide(10000)
        }).rename(["EVI"])
    savi= image.expression("((NIR-ROUGE) / (NIR + ROUGE + 0.5))*(1 + 0.5)",{
            "NIR":image.select("B8"),
            "ROUGE":image.select("B4")
        }).rename(["SAVI"])
    bsi = image.expression(
    '((RED + SWIR) - (NIR + BLUE)) / ((RED + SWIR) + (NIR + BLUE)) ',
    {
            'RED': image.select('B4'),
            'BLUE': image.select('B2'),
            'NIR': image.select('B8'),
            'SWIR': image.select('B11')

        }
    ) \
    .rename('BSI')
    return image.addBands(ndvi).addBands(ndwi).addBands(evi) \
    .addBands(savi).addBands(bsi).copyProperties(image,['"system":time_start'])

allIndex = func_azz

#Collection with all images also containing the indices
collection = s2.select(inbands).map(allIndex)

# Define and calculate the median bands and the other index statistics
band_median = collection.select(inbands).median()
ndvimax = collection.select('NDVI').reduce(ee.Reducer.max()).rename("NDVI_MAX")
ndvimean = collection.select('NDVI').reduce(ee.Reducer.mean()).rename("NDVI")
ndvistd = collection.select('NDVI').reduce(ee.Reducer.stdDev()).float().rename("NDVI_STD")
savimax = collection.select('SAVI').reduce(ee.Reducer.max()).rename("SAVI_MAX")
savimean = collection.select('SAVI').reduce(ee.Reducer.mean()).rename("SAVI")
savistd = collection.select('SAVI').reduce(ee.Reducer.stdDev()).float().rename("SAVI_STD")
ndwimax = collection.select('NDWI').reduce(ee.Reducer.max()).rename("NDWI_MAX")
ndwimean = collection.select('NDWI').reduce(ee.Reducer.mean()).rename("NDWI")
ndwistd = collection.select('NDWI').reduce(ee.Reducer.stdDev()).float().rename("NDWI_STD")
evimax = collection.select('EVI').reduce(ee.Reducer.max()).rename("EVI_MAX")
evimean = collection.select('EVI').reduce(ee.Reducer.mean()).rename("EVI")
evistd = collection.select('EVI').reduce(ee.Reducer.stdDev()).float().rename("EVI_STD")
#print(band_median.getInfo())
#Add the index statistics to the median bands and clip the dataset with the ROI
raster_survey = band_median.addBands(ndvimax).addBands(ndvimean).addBands(ndvistd) \
.addBands(ndwimax).addBands(ndwimean).addBands(ndwistd) \
.addBands(savimax).addBands(savimean).addBands(savistd) \
.addBands(evimax).addBands(evimean).addBands(evistd) \
.clip(roi)


In [38]:
size_segmentation = 15

#----- Define the GLCM indices used in input for the PCA
glcm_bands= ["gray_asm","gray_contrast","gray_corr","gray_ent","gray_var","gray_idm","gray_savg"]

#-----Object-based Approach

# Segmentation using a SNIC approach based on the dataset previosly generated
seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation)

#set the bands names to snic computing
snic_bands = ["NDVI_MAX","NDVI","NDVI_STD",
"NDWI_MAX","NDWI","NDWI_STD",
"SAVI_MAX","SAVI","SAVI_STD",
"B4","EVI_MAX","EVI","B8"
]
snic = ee.Algorithms.Image.Segmentation.SNIC(
image=raster_survey.select(snic_bands),
compactness =0.1,
connectivity=8,
neighborhoodSize=256,
seeds=seeds
)
#Create and rescale a grayscale image for GLCM ( Enfin de compte c'est une ponderation de bandes pour avoir une image en fonction de ces bandes)
#Pour mon experience donc, je vais choisir d'utiliser soir EVI ,soit SAVI , ces indices prenneen ten compte des corrections au niveaux des aerosol
# atmo et la reflectance du sol

# the glcmTexture size (in pixel) can be adjusted considering the spatial resolution and the object textural characteristics
gray = raster_survey.select('EVI_MAX').rename('gray').unitScale(-1,1).multiply(255).uint8()
glcm = gray.glcmTexture(size= 2)
glcm


In [49]:
# Set up the min and max value of the gray band image
scale = 10
geom = roi
minMax = gray.reduceRegion(
reducer=ee.Reducer.minMax(),
geometry =roi,
scale =10,
bestEffort=True
)
gray_min = minMax.get('gray_min')
gray_max = minMax.get('gray_max')
#print("La valeur maximale de la bande "gray": ",ee.Number(gray_max),"  la valeur doit s'etaler de 0-255".getInfo())
#print("La valeur minimale de la bande "gray": ",ee.Number(gray_min),"Plus la valeur s'etire plus, l'algorithme glcm est bon".getInfo())

#----- Define the GLCM indices used in input for the PCA
glcm_bands= ["gray_asm","gray_contrast","gray_corr","gray_ent","gray_var","gray_idm","gray_savg"]

#print("Liste de bandes issues des resultats obtenus "glcm": ",glcm.bandNames().getInfo())
#####################PCA METHODS###################
#--- Before the PCA the glcm bands are scaled
image = glcm.select(glcm_bands)
# calculate the min and max value of an image
minMax = image.reduceRegion(
reducer=ee.Reducer.minMax(),
geometry=roi,
scale=10,
maxPixels=1e13,
bestEffort =True
)

# Normalize the data
# First define the normalization function
def normalization(name):
    name = ee.String(name)
    band = image.select(name)
    min_val = ee.Number(minMax.get(name.cat('_min')))
    max_val = ee.Number(minMax.get(name.cat('_max')))
    return band.subtract(min_val).divide(max_val.subtract(min_val))

# Then apply it to create an image collection
glcm = ee.ImageCollection.fromImages(
    image.bandNames().map(normalization)
).toBands().rename(image.bandNames())

#---- Apply the PCA
# The code relating to the PCA was adapted from the GEE documentation  "https":#developers.google.com/earth-engine/guides/arrays_eigen_analysis

# Get some information about the input to be used later.

bandNames = glcm.bandNames()

# Mean center the data to enable a faster covariance reducer and an SD stretch of the principal components.
meanDict = glcm.reduceRegion(
reducer=ee.Reducer.mean(),
geometry=roi,
scale=10,
maxPixels=1e16,
bestEffort=True
)

#,On doit transofmer sous forme une image constante les valeurs moyennées pur chaque bands
#"Donc"::::
means = ee.Image.constant(meanDict.values(bandNames))
#soustraction par la moyenne de l 'image constante (band math)
centered = glcm.subtract(means)

# This helper function returns a list of new band names.

def getNewBandNames(prefix):
    seq = ee.List.sequence(1, bandNames.length())

    def func_unn(b):
            return ee.String(prefix).cat(ee.Number(b).int())

    return seq.map(func_unn)

def getPrincipalComponents(centered, scale):
    # Collapse the bands of the image into a 1D array per pixel.
    arrays = centered.toArray()

    # Compute the covariance of the bands within the region.
    covar = arrays.reduceRegion(
        reducer=ee.Reducer.centeredCovariance(),
        geometry=roi,
        scale=10,
        bestEffort =True,
        maxPixels =1e13
        )

    # Get the 'array' covariance result and cast to an array.
    # This represents the band-to-band covariance within the region.
    covarArray = ee.Array(covar.get('array'))

    # Perform an eigen analysis and slice apart the values and vectors.
    eigens = covarArray.eigen()

    # This is a P-length vector of Eigenvalues.
    eigenValues = eigens.slice(1, 0, 1)
    # This is a PxP matrix with eigenvectors in rows.
    eigenVectors = eigens.slice(1, 1)

    # Convert the array image to 2D arrays for matrix computations.
    arrayImage = arrays.toArray(1)

    # Left multiply the image array by the matrix of eigenvectors.
    principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

    # Turn the square roots of the Eigenvalues into a P-band image.
    sdImage = ee.Image(eigenValues.sqrt()) \
    .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

    # Turn the PCs into a P-band image, normalized by SD.
    return principalComponents \
    .arrayProject([0]) \
    .arrayFlatten([getNewBandNames('pc')]) \
    .divide(sdImage)

# Get the PCs at the specified scale and in the specified region
pcImage = getPrincipalComponents(centered, 10)

#Select the band "clusters" from the snic output fixed on its scale of 10 meters and add them the PC1 taken from the PCA data.
# Calculate the mean for each segment with respect to the pixels in that cluster
clusters_snic = snic.select("clusters")
#clusters_snic = clusters_snic.reproject (**{"crs": clusters_snic.projection (), "scale": 10})
#Map.addLayer(clusters_snic.randomVisualizer(), {}, 'clusters')

new_feature = clusters_snic.addBands(pcImage.select("pc1"))

new_feature_mean =snic.reduceConnectedComponents(
reducer=ee.Reducer.mean(),
labelBand='clusters'
)

###### Second method is to compare our model with GLCM methout , without it##/
#SSSSSSOOOOOOOOOOOOOOO
reducer_snic = snic.reduceConnectedComponents(
reducer =ee.Reducer.mean(),
labelBand ="clusters"
)

#Create a dataset with the new band used so far together with the band "clusters" and their new mean parameters
final_bands =new_feature_mean.addBands(snic)


In [47]:
Map.addLayer(final_bands)
Map

Map(bottom=1141642.0, center=[-15.772761158977673, 46.1202621459961], controls=(WidgetControl(options=['positi…